# **Classificação de imagens raio-x utilizando Deep Learning**

Autor(a): Maria Eduarda Ornelas Hisse

Orientador(a): Prof(a). Dra. Sílvia Cristina Dias Pinto

Descrição: Algoritmo desenvolvido durante o Trabalho de Conclusão de Curso apresentado como pré-requisito para obtenção do título de Engenheiro de Computação, ao Departamento de Modelagem Computacional, do Instituto Politécnico, da Universidade do Estado do Rio de Janeiro.

Objetivo: Realizar os experimentos e a obtenção dos resultados para a classificação de imagens de radiografias de tórax utilizando o conjunto de dados com uma abordagem de duas classes (e posteriormente a classe Doente será classificada em três outras classes) será  e redes neurais convolucionais.

### *Importando as bibliotecas necessárias*

In [ ]:
!pip install opendatasets

In [ ]:
!pip install iteration_utilities

In [ ]:
import os
import cv2
import random
import sklearn
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import opendatasets as op
from tensorflow import keras
import matplotlib.pyplot as plt
from iteration_utilities import duplicates
from sklearn.model_selection import cross_validate
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.metrics import roc_curve, auc, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report

### *Baixando a base de dados*

In [ ]:
op.download("https://www.kaggle.com/datasets/tawsifurrahman/covid19-radiography-database/")

## **Lendo a base de dados**

In [ ]:
class config:
    seed = 42
    img_size = [256,256]
    msk_mode = 'grayscale'
    img_mode = 'grayscale'
    msk_channels = 1
    img_channels = 1
    img_type_num = 0
    msk_type_num = 0
    backbone = None
    activation = None
    batch_size = 16
    train_epochs = 10
    lr = 1e-4

In [ ]:
path_df = pd.DataFrame(columns=['img_path','msk_path','img_shape','msk_shape','class','subclass'])
for cat in ['COVID','Lung_Opacity','Normal','Viral Pneumonia']:
    dir_ = f"covid19-radiography-database/COVID-19_Radiography_Dataset/{cat}"
    for f in os.listdir(f"{dir_}/images"):
        s1 = cv2.imread(f"{dir_}/images/{f}",config.img_type_num).shape
        s2 = cv2.imread(f"{dir_}/masks/{f}",config.msk_type_num).shape
        subcat = cat
        if cat == 'Lung_Opacity' or cat == 'COVID' or cat =='Viral Pneumonia': cat = 'Doente'
        dic = {'img_path':f"{dir_}/images/{f}",'msk_path':f"{dir_}/masks/{f}",'img_shape':s1,'msk_shape':s2,'class': cat,'subclass': subcat}
        path_df = path_df.append(dic,ignore_index=True)

In [ ]:
path_df

### *Visualizando as imagens raio-x de cada classe*

O conjunto de dados é composto por imagens de raio-x que estão classificadas em quatro classes: Covid-19, Normal, Pneumonia Viral e Lung_Opacity (Infecção Pulmonar não classificada como Covid-19).

In [ ]:
labels = ['COVID','Lung_Opacity','Normal','Viral Pneumonia']
for i in range(4):
    fig, axs = plt.subplots(1,5, figsize = (50,10))
    print('Imagens raio-x da classe', labels[i])
    paths = [k for k in path_df['img_path'] if(labels[i] in k)]
    for j in range(5):
        img = cv2.imread(paths[j])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        axs[j].imshow(img)
    plt.show()

## **Pré-processamento da base de dados**

Originalmente, as imagens são fornecidas em .png com dimensões 299 x 299, passaram pelo corte das áreas superiores e inferiores obtendo as dimensões 256 x 256.

In [ ]:
def processing(image, mask):

  # Lê a imagem e converte para RBG
  image = cv2.imread(image)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  plt.imshow(image)
  plt.show()

  # Redimensionando a imagem para a dimensão da máscara 256 x 256
  image_res = cv2.resize(image, (256, 256))
  plt.imshow(image_res)
  plt.show()

  # Lê a máscara correspondente a imagem e converte para RGB
  mask = cv2.imread(mask)
  mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
  plt.imshow(mask)
  plt.show()

  # Multiplicando a máscara e a imagem
  mask_mult = mask // 255
  mask_mult = image_res * mask_mult
  plt.imshow(mask_mult)
  plt.show()

  # Invertendo a máscara
  mask_inv = mask.copy()
  for index, i in np.ndenumerate(mask):
    if i == 0: mask_inv[index] = 255
    else: mask_inv[index] = 0
  plt.imshow(mask_inv)
  plt.show()

  # Multiplicando a máscara invertida e a imagem
  mask_inv_mult = mask_inv // 255
  mask_inv_mult = image_res * mask_inv_mult
  plt.imshow(mask_inv_mult)
  plt.show()

In [ ]:
# Lendo uma imagem da base de dados
image = path_df['img_path'][0]

# Lendo a máscara correspondente a imagem
mask = path_df['msk_path'][0]

processing(image, mask)

### *Pré-processamento*

*   Redimensionar a imagem para 256 x 256 pixels
*   Multiplicar a imagem e máscara



In [ ]:
def pre_processing(img, mask):
  # Redimensionando a imagem
  img = tf.image.resize(img, (256, 256))

  # Multiplicando a máscara e a imagem
  mask = mask / 255.0
  mask_mult = img * mask

  return img, mask_mult

In [ ]:
image = path_df['img_path'][0]
mask = path_df['msk_path'][0]

img = keras.preprocessing.image.img_to_array(keras.preprocessing.image.load_img(image))
mask = keras.preprocessing.image.img_to_array(keras.preprocessing.image.load_img(mask))
img, mask = pre_processing(img, mask)

mask = np.asarray(mask)
mask = mask.astype(np.uint8)
plt.imshow(mask[:,:,0], cmap="gray")
plt.show()

## **Balanceamento da base de dados**

Rebalanceamento dos dados para garantir que cada lote seja balanceado em termos de classes (cross-validation, treinamento, teste e validação).

Técnica *Under-Sampling* reduz o desbalanceamento do dataset focando na classe majoritária. Ou seja, elimina aleatoriamente entradas da classe com maior número de ocorrências.

### *Distribuição da base de dados antes do balanceamento*

In [ ]:
images = np.asarray(path_df['img_path'])
classes = np.asarray(path_df['class'])

# Ver o balanceamento das classes
print(pd.Series(classes).value_counts())

# Plotar a nova distribuição de classes
sns.countplot(x=classes)

### *Distribuição da base de dados após o balanceamento*

In [ ]:
# Balanceando a base de dados para a classificação das classes através da técnica under-slamping
def balanced_under_sampler():
  rus = RandomUnderSampler(replacement = False)
  x_res, y_res = rus.fit_resample(images.reshape(-1, 1), classes.reshape(-1, 1))

  x_res = x_res.reshape(-1)
  return x_res, y_res

In [ ]:
# Balanceando a base de dados para a classificação das classes escolhendo aleatoriamente as imagens utilizadas
def balanced_random():
  x_res = []
  y_res = []

  for cat in ['COVID','Lung_Opacity','Normal','Viral Pneumonia']:
    dir_ = f"covid19-radiography-database/COVID-19_Radiography_Dataset/{cat}"
    path = os.listdir(f"{dir_}/images/")
    list_numbers = random.sample(range(len(path)), pd.Series(classes).value_counts().min())
    for i in list_numbers:
        x_res.append(f"{dir_}/images/{path[i]}")
        y_res.append(cat)
  return x_res, y_res

## **Dividindo a base de dados em treinamento, teste e validação**

A base de dados foi dividida em três conjuntos de treinamento, teste e validação. Sendo 70% da base destinada a treinamento do modelo de classificação, 10% a validação e 20% a teste.

In [ ]:
def divide_base_date(x_res, segmentacao):
  # Embaralhando a base de dados
  random.shuffle(x_res)

  # Conjunto de treinamento
  X_train = []
  y_train = []
  y_train_doente = []
  x_res = x_res[1:10000]
  for train in x_res[0:int(len(x_res)*0.7)]:
    img = keras.preprocessing.image.img_to_array(keras.preprocessing.image.load_img(train))
    img, mask = pre_processing(img, mask)
    mask = keras.preprocessing.image.img_to_array(keras.preprocessing.image.load_img(train.replace("images", "masks")))
    if segmentacao:
      X_train.append(np.array(mask, dtype=np.float16))
    else:
      X_train.append(np.array(img, dtype=np.float16))
    classe = train.split('/')[-3]
    y_train_doente.append(classe)
    if classe == 'Lung_Opacity' or classe == 'COVID' or classe =='Viral Pneumonia': classe = 'Doente'
    y_train.append(classe)

  # Conjunto de validação
  X_val = []
  y_val = []
  y_val_doente = []
  for val in x_res[int(len(x_res)*0.7):int(len(x_res)*0.8)]:
    img = keras.preprocessing.image.img_to_array(keras.preprocessing.image.load_img(val))
    mask = keras.preprocessing.image.img_to_array(keras.preprocessing.image.load_img(val.replace("images", "masks")))
    img, mask = pre_processing(img, mask)
    if segmentacao:
      X_val.append(np.array(mask, dtype=np.float16))
    else:
      X_val.append(np.array(img, dtype=np.float16))
    classe = val.split('/')[-3]
    y_val_doente.append(classe)
    if classe == 'Lung_Opacity' or classe == 'COVID' or classe =='Viral Pneumonia': classe = 'Doente'
    y_val.append(classe)

  # Conjunto de teste
  X_test = []
  y_test = []
  y_test_doente = []
  for test in x_res[int(len(x_res)*0.8):int(len(x_res))]:
    img = keras.preprocessing.image.img_to_array(keras.preprocessing.image.load_img(test))
    mask = keras.preprocessing.image.img_to_array(keras.preprocessing.image.load_img(test.replace("images", "masks")))
    img, mask = pre_processing(img, mask)
    if segmentacao:
      X_test_mask.append(np.array(mask, dtype=np.float16))
    else:
      X_test.append(np.array(img, dtype=np.float16))
    classe = test.split('/')[-3]
    y_test_doente.append(classe)
    if classe == 'Lung_Opacity' or classe == 'COVID' or classe =='Viral Pneumonia': classe = 'Doente'
    y_test.append(classe)

  return X_train, X_val, X_test, X_train_mask, X_val_mask, X_test_mask, y_train, y_val, y_test, y_train_doente, y_val_doente, y_test_doente

In [ ]:
X_train, X_val, X_test, X_train_mask, X_val_mask, X_test_mask, y_train, y_val, y_test, y_train_doente, y_val_doente, y_test_doente = divide_base_date(images, 0)

### Número de imagens de cada conjunto

In [ ]:
print('Quantidade de imagens do conjunto de treinamento:', len(y_train))
print('Quantidade de imagens do conjunto de validação:', len(y_val))
print('Quantidade de imagens do conjunto de teste:', len(y_test))

### *Etiquetas de codificação*

In [ ]:
encoder = LabelEncoder()
y_train_enconder = encoder.fit_transform(y_train)
y_val_enconder = encoder.fit_transform(y_val)
y_test_enconder = encoder.fit_transform(y_test)

classes = list(encoder.classes_)
print(classes)

['Doente', 'Normal']


### *Transformando o conjunto de dados em um array*

In [ ]:
X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)

In [ ]:
X_train_mask = np.array(X_train_mask)
X_val_mask = np.array(X_val_mask)
X_test_mask = np.array(X_test_mask)

## **ResNet-50**

In [ ]:
def ResNet50(len_inputs, len_classes):
  baseModel = keras.applications.ResNet50V2(weights = 'imagenet', include_top = False, input_shape = len_inputs)

  for layer in baseModel.layers: layer.trainable=False

  headModel = baseModel.output
  headModel = keras.layers.Dropout(0.5)(headModel)
  headModel = keras.layers.Flatten()(headModel)
  headModel = keras.layers.BatchNormalization()(headModel)

  headModel = keras.layers.Dense(2048, activation = "relu")(headModel)
  headModel = keras.layers.BatchNormalization()(headModel)
  headModel = keras.layers.Dropout(0.5)(headModel)
  headModel = keras.layers.Dense(1024, activation = "relu")(headModel)
  headModel = keras.layers.BatchNormalization()(headModel)
  headModel = keras.layers.Dropout(0.5)(headModel)

  headModel = keras.layers.Dense(len_classes, activation = "softmax")(headModel)
  model = keras.models.Model(inputs = baseModel.input, outputs = headModel)

  return model

## **ResNet-101**

In [ ]:
def ResNet101(len_inputs, len_classes):
  baseModel = keras.applications.ResNet101V2(weights = 'imagenet', include_top = False, input_shape = len_inputs)

  for layer in baseModel.layers: layer.trainable = False

  headModel = baseModel.output
  headModel = keras.layers.Dropout(0.5)(headModel)
  headModel = keras.layers.Flatten()(headModel)
  headModel = keras.layers.BatchNormalization()(headModel)

  headModel = keras.layers.Dense(2048, activation = "relu")(headModel)
  headModel = keras.layers.BatchNormalization()(headModel)
  headModel = keras.layers.Dropout(0.5)(headModel)
  headModel = keras.layers.Dense(1024, activation = "relu")(headModel)
  headModel = keras.layers.BatchNormalization()(headModel)
  headModel = keras.layers.Dropout(0.5)(headModel)

  headModel = keras.layers.Dense(len_classes, activation = "softmax")(headModel)
  model = keras.models.Model(inputs = baseModel.input, outputs = headModel)

  return model

## **VGG-16**

In [ ]:
def VGG16(len_inputs, len_classes):
    baseModel = keras.applications.VGG16(weights = 'imagenet', include_top = False, input_shape = len_inputs)

    for layer in baseModel.layers: layer.trainable = False

    headModel = baseModel.output
    headModel = keras.layers.GlobalAveragePooling2D()(headModel)
    headModel = keras.layers.BatchNormalization()(headModel)

    headModel = keras.layers.Dense(128, activation="relu")(headModel)
    headModel = keras.layers.BatchNormalization()(headModel)
    headModel = keras.layers.Dense(64, activation="relu")(headModel)
    headModel = keras.layers.BatchNormalization()(headModel)

    headModel = keras.layers.Dense(len_classes, activation="softmax")(headModel)
    model = keras.models.Model(inputs = baseModel.input, outputs = headModel)

    return model

## **VGG-19**

In [ ]:
def VGG19(len_inputs, len_classes):
    baseModel = keras.applications.VGG19(weights= 'imagenet', include_top = False, input_shape = len_inputs)

    for layer in baseModel.layers: layer.trainable = False

    headModel = baseModel.output
    headModel = keras.layers.GlobalAveragePooling2D()(headModel)
    headModel = keras.layers.BatchNormalization()(headModel)

    headModel = keras.layers.Dense(128, activation="relu")(headModel)
    headModel = keras.layers.BatchNormalization()(headModel)
    headModel = keras.layers.Dense(64, activation="relu")(headModel)
    headModel = keras.layers.BatchNormalization()(headModel)

    headModel = keras.layers.Dense(len_classes, activation="softmax")(headModel)
    model = keras.models.Model(inputs = baseModel.input, outputs = headModel)

    return model

## **Xception**

In [ ]:
def Xception(len_inputs, len_classes):
  baseModel = keras.applications.Xception(weights = 'imagenet', include_top = False, input_shape = len_inputs)

  for layer in baseModel.layers: layer.trainable = False

  headModel = baseModel.output
  headModel = keras.layers.GlobalAveragePooling2D(name = 'avg_pool')(headModel)
  headModel = keras.layers.Dropout(0.75)(headModel)
  headModel = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.01, center=True, scale=True, beta_initializer="zeros", gamma_initializer="ones", moving_mean_initializer="zeros", moving_variance_initializer="ones")(headModel)

  headModel = keras.layers.Dense(len_classes, activation = 'softmax')(headModel)
  model = keras.models.Model(inputs = baseModel.input, outputs = headModel)

  return model


## **DenseNet-121**

In [ ]:
def DenseNet121(len_inputs, len_classes):
  baseModel = keras.applications.DenseNet121(weights = 'imagenet', include_top = False, input_shape = len_inputs)

  for layer in baseModel.layers: layer.trainable = False

  headModel = baseModel.output
  headModel = keras.layers.Dropout(0.3)(headModel)
  headModel = keras.layers.Flatten()(headModel)
  headModel = keras.layers.Dense(64, activation='tanh',kernel_initializer=keras.initializers.GlorotNormal(),bias_regularizer=tf.keras.regularizers.L2(0.0001), kernel_regularizer=tf.keras.regularizers.L2(0.0001), activity_regularizer = tf.keras.regularizers.L2(0.0001))(headModel)
  headModel = keras.layers.Dropout(0.3)(headModel)
  headModel = keras.layers.Dense(32, activation='tanh',kernel_initializer=keras.initializers.GlorotNormal(),bias_regularizer=tf.keras.regularizers.L2(0.0001) ,kernel_regularizer=tf.keras.regularizers.L2(0.0001), activity_regularizer = tf.keras.regularizers.L2(0.0001))(headModel)
  headModel = keras.layers.Dropout(0.3)(headModel)
  headModel = keras.layers.Dense(16, activation='tanh',kernel_initializer=keras.initializers.GlorotNormal(),bias_regularizer=tf.keras.regularizers.L2(0.0001) ,kernel_regularizer=tf.keras.regularizers.L2(0.0001), activity_regularizer = tf.keras.regularizers.L2(0.0001))(headModel)
  headModel = keras.layers.Dropout(0.3)(headModel)

  headModel = keras.layers.Dense(len_classes, activation='softmax', kernel_initializer=keras.initializers.GlorotNormal(), bias_regularizer=tf.keras.regularizers.L2(0.0001),kernel_regularizer=tf.keras.regularizers.L2(0.0001), activity_regularizer = tf.keras.regularizers.L2(0.0001))(headModel)
  model = keras.models.Model(inputs = baseModel.input, outputs = headModel)

  return model

## **MobileNetV2**

In [ ]:
def MobileNetV2(len_inputs, len_classes):
  baseModel = keras.applications.MobileNetV2(weights = 'imagenet', include_top = False, input_shape = len_inputs)

  for layer in baseModel.layers: layer.trainable = False

  headModel = baseModel.output
  headModel = keras.layers.Flatten()(headModel)
  headModel = keras.layers.Dense(1024, activation='relu')(headModel)
  headModel = keras.layers.BatchNormalization()(headModel)
  headModel = keras.layers.Dense(512, activation='relu')(headModel)
  headModel = keras.layers.BatchNormalization()(headModel)
  headModel = keras.layers.Dense(128, activation='relu')(headModel)
  headModel = keras.layers.Dropout(0.15)(headModel)
  headModel = keras.layers.BatchNormalization()(headModel)
  headModel = keras.layers.Dense(64, activation='relu')(headModel)
  headModel = keras.layers.Dropout(0.3)(headModel)
  headModel = keras.layers.BatchNormalization()(headModel)

  headModel = keras.layers.Dense(len_classes, activation="softmax")(headModel)
  model = keras.models.Model(inputs = baseModel.input, outputs = headModel)

  return model

## **InceptionV3**

In [ ]:
def InceptionV3(len_inputs, len_classes):
  baseModel = keras.applications.InceptionV3(weights = 'imagenet', include_top = False, input_shape = len_inputs)

  for layer in baseModel.layers: layer.trainable = False

  headModel = baseModel.output
  headModel = keras.layers.GlobalAveragePooling2D()(headModel)
  headModel = keras.layers.Dense(1024, activation='relu')(headModel)
  headModel = keras.layers.BatchNormalization()(headModel)
  headModel = keras.layers.Dense(512, activation='relu')(headModel)
  headModel = keras.layers.BatchNormalization()(headModel)
  headModel = keras.layers.Dense(128, activation='relu')(headModel)
  headModel = keras.layers.Dropout(0.15)(headModel)
  headModel = keras.layers.BatchNormalization()(headModel)
  headModel = keras.layers.Dense(64, activation='relu')(headModel)
  headModel = keras.layers.Dropout(0.3)(headModel)
  headModel = keras.layers.BatchNormalization()(headModel)

  headModel = keras.layers.Dense(len_classes, activation="softmax")(headModel)
  model = keras.models.Model(inputs = baseModel.input, outputs = headModel)

  return model

## **Escolha do modelo**

In [ ]:
def model_classifier(model, len_inputs, len_classes):
  if model == "vgg16":
    model = VGG16(len_inputs, len_classes)
  elif model == "vgg19":
    model = VGG19(len_inputs, len_classes)
  elif model == "resnet50":
    model = ResNet50(len_inputs, len_classes)
  elif model == "resnet101":
    model = ResNet101(len_inputs, len_classes)
  elif model == "xception":
    model = Xception(len_inputs, len_classes)
  elif model == "densenet121":
    model = DenseNet121(len_inputs, len_classes)
  elif model == "mobilenetv2":
    model = MobileNetV2(len_inputs, len_classes)
  elif model == "inceptionv3":
    model = InceptionV3(len_inputs, len_classes)
  else:
    model = -1

  return model

In [ ]:
classifier = "inceptionv3"
len_inputs = (256, 256, 3)
len_classes = 4
learning_rate = 0.00001
loss = 'sparse_categorical_crossentropy'
optimizers = keras.optimizers.Adam(learning_rate = learning_rate)

In [ ]:
model = model_classifier(classifier, len_inputs, len_classes)

In [ ]:
model.compile(loss = loss, optimizer = optimizers, metrics = ['accuracy'])

In [ ]:
print('Sumário do modelo:', classifier)
model.summary()

## **Treinamento do modelo e salvando em cada época**

Aqui, o modelo é salvo em cada época em que val_accuracy for maior que seus valores anteriores. Então, esse modelo pode ser usado mais tarde para uma melhor generalização.

In [ ]:
def model_fit(checkpoint_path):
  early_stopping = keras.callbacks.EarlyStopping(monitor='loss', patience = 3)
  reduce_lr =  keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001)
  learning_rate_reduction = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', patience=2, verbose=2, factor=0.5, min_lr=0.00001)

  checkpoint_dir = os.path.dirname(checkpoint_path)
  model_check = keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True, monitor = 'val_accuracy', mode = 'max', save_best_only=True, verbose=1)

  cp_callback = [early_stopping, reduce_lr , learning_rate_reduction, model_check]

  return cp_callback

In [ ]:
epochs = 40
batch_size = 32

In [ ]:
model_history = model.fit(X_train, y_train_enconder, epochs = epochs, batch_size = batch_size, validation_data = (X_val, y_val_enconder))

## **Gráficos de Accuracy, Val_Accuracy, Loss e Val_Loss**

In [ ]:
model_history.history

In [ ]:
epochs = len(model_history.history['accuracy'])

In [ ]:
fig, axs = plt.subplots(1,2, figsize = (20,7))
axs[0].plot(np.arange(1, epochs + 1), model_history.history['accuracy'], label = 'Accuracy')
axs[0].plot(np.arange(1, epochs + 1), model_history.history['val_accuracy'], label = 'Val Accuracy')
axs[0].legend()
axs[0].grid()

axs[1].plot(np.arange(1, epochs + 1), model_history.history['loss'], label = 'Loss')
axs[1].plot(np.arange(1, epochs + 1), model_history.history['val_loss'], label = 'Val Loss')
axs[1].legend()
axs[1].grid()

plt.title("Modelo " + classifier)
plt.show()

## **Predições**

In [ ]:
prob_pred = model.predict(X_test)
y_pred = np.argmax(prob_pred, axis=1)

### *Acurácia no Conjunto de Teste*

In [ ]:
print('Acurácia no Conjunto de Teste:', accuracy_score(y_test_enconder, y_pred))
print('Relatório de Classificação:')
print(classification_report(encoder.inverse_transform(y_test_enconder), encoder.inverse_transform(y_pred), zero_division = 1))

### *Avaliação no conjunto de teste*

Retorna o valor de perda e os valores de métricas para o modelo no modo de teste.

In [ ]:
evalute = model.evaluate(X_test, y_test_enconder)

In [ ]:
print("Accuracy: {:.2f}%".format(evalute[1] * 100))
print("Loss: {}".format(evalute[0]))

### *Matriz de Confusão*

In [ ]:
cm = confusion_matrix(y_test_enconder, y_pred)

In [ ]:
cm

In [ ]:
plt.figure(figsize=(11,7))
x_axis_labels = classes
y_axis_labels = classes
plt.rcParams.update({'font.size': 25})
sns.heatmap(cm, xticklabels = x_axis_labels, yticklabels = y_axis_labels, annot= True, fmt='',cmap = 'Blues')
plt.xlabel('Predição', fontsize = 20)
plt.ylabel("Real", fontsize = 20)
plt.title('Matriz de confusão ')
plt.show()

## **Mapa de Calor (Grad-CAM)**

In [ ]:
def get_img_array(img_path, size):
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    array = keras.preprocessing.image.img_to_array(img)
    array = np.expand_dims(array, axis=0)
    return array

In [ ]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = keras.models.Model([model.inputs], [model.get_layer(last_conv_layer_name).output, model.output])

    # Compita-se o gradiente da classe superior prevista para nossa imagem de entrada em relação às ações da última camada de conversão
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # Este é o gradiente do neurônio de saída (top previsto ou escolhido) em relação ao mapa de recursos de saída da última camada de conversão
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # Este é um vetor onde cada entrada é a intensidade média do gradiente sobre um canal específico do mapa de recursos
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # Multiplica-se cada canal na matriz do mapa de recursos por "quão importante é esse canal" em relação à classe mais alta prevista
    # Em seguida, somamos todos os canais para obter a ativação da classe do mapa de calor
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # Para fins de visualização, também normalizaremos o mapa de calor entre 0 e 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [ ]:
def save_and_display_gradcam(img, heatmap, cam_path="cam.jpg", alpha=0.4):
    # Redimensiona-se o mapa de calor para um intervalo de 0 a 255
    heatmap = np.uint8(255 * heatmap)

    # Usa-se o mapa de cores do jato para colorir o mapa de calor
    jet = plt.get_cmap("jet")

    # Use valores RGB do mapa de cores
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Cria-se uma imagem com mapa de calor colorido RGB
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Sobrepõe-se o mapa de calor na imagem original
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    # Salva-se a imagem sobreposta
    superimposed_img.save(cam_path)

    return cam_path

In [ ]:
preprocess_input = keras.applications.mobilenet_v2.preprocess_input
decode_predictions = keras.applications.mobilenet_v2.decode_predictions

last_conv_layer_name = 'conv2d_93'

# Remove-se o softmax da última camada
model.layers[-1].activation = None

In [ ]:
# Exibe a parte das imagens usadas pela rede neural para classificar as imagens
fig, axes = plt.subplots(nrows = 4, ncols = 4, figsize = (20, 20), subplot_kw = {'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    img_array = X_test[i]
    heatmap = make_gradcam_heatmap(img_array.reshape(1, 256, 256, 3), model, last_conv_layer_name)
    cam_path = save_and_display_gradcam(img_array, heatmap, "cam.jpg", 0.8)
    class_test = classes[y_test_enconder[i]]
    class_prediction = classes[y_pred[i]]
    ax.imshow(plt.imread(cam_path))
    ax.set_title(f"True: {class_test}\nPredicted: {class_prediction}", fontsize = 14)
plt.tight_layout()
plt.show()

## **Carregando pesos de modelo do ponto de verificação e reavaliar**

## **Três Classes**

Classificação da classe Doente em três subclasses: Covid, Viral Pneumonia e Lung_Opacity.

In [ ]:
X_test_doente = []
y_test_doente2 = []
for i in range(len(y_test)):
  if y_test[i] == 'Doente':
    X_test_doente.append(X_test[i])
    y_test_doente2.append(y_test_doente[i])

X_val_doente = []
y_val_doente2 = []
for i in range(len(y_val)):
  if y_val[i] == 'Doente':
    X_val_doente.append(X_val[i])
    y_val_doente2.append(y_val_doente[i])

X_train_doente = []
y_train_doente2 = []
for i in range(len(y_train)):
  if y_train[i] == 'Doente':
    X_train_doente.append(X_train[i])
    y_train_doente2.append(y_train_doente[i])

In [ ]:
print('Quatidade de imagens do conjunto de treinamento:', len(y_train_doente2))
print('Quatidade de imagens do conjunto de validação:', len(y_val_doente2))
print('Quatidade de imagens do conjunto de teste:', len(y_test_doente2))

In [ ]:
encoder = LabelEncoder()
y_train_doente2 = encoder.fit_transform(y_train_doente2)
y_val_doente2 = encoder.fit_transform(y_val_doente2)
y_test_doente2 = encoder.fit_transform(y_test_doente2)

X_train_doente = np.array(X_train_doente)
X_val_doente = np.array(X_val_doente)
X_test_doente = np.array(X_test_doente)

classes_doente = list(encoder.classes_)
print(classes_doente)

In [ ]:
model_history = model.fit(X_train_doente, y_train_doente2, epochs = epochs, batch_size = batch_size, validation_data = (X_val_doente, y_val_doente2))

In [ ]:
model_history.history

In [ ]:
fig, axs = plt.subplots(1,2, figsize = (20,7))
axs[0].plot(np.arange(1, epochs + 1), model_history.history['accuracy'], label = 'Accuracy')
axs[0].plot(np.arange(1, epochs + 1), model_history.history['val_accuracy'], label = 'Val Accuracy')
axs[0].legend()
axs[0].grid()

axs[1].plot(np.arange(1, epochs + 1), model_history.history['loss'], label = 'Loss')
axs[1].plot(np.arange(1, epochs + 1), model_history.history['val_loss'], label = 'Val Loss')
axs[1].legend()
axs[1].grid()

plt.title("Modelo " + classifier)
plt.show()

In [ ]:
prob_pred_doente = model.predict(X_test_doente)
y_pred_doente = np.argmax(prob_pred_doente, axis=1)

In [ ]:
print('Acurácia no Conjunto de Teste:', accuracy_score(y_test_doente2, y_pred_doente))
print('Relatório de Classificação:')
print(classification_report(encoder.inverse_transform(y_test_doente2), encoder.inverse_transform(y_pred_doente), zero_division = 1))

In [ ]:
cm = confusion_matrix(y_test_doente2, y_pred_doente)

In [ ]:
plt.figure(figsize=(11,7))
x_axis_labels = classes_doente
y_axis_labels = classes_doente
plt.rcParams.update({'font.size': 25})
sns.heatmap(cm, xticklabels = x_axis_labels, yticklabels = y_axis_labels, annot= True, fmt='',cmap = 'Blues')
plt.xlabel('Predição', fontsize = 20)
plt.ylabel("Real", fontsize = 20)
plt.title('Matriz de confusão ')
plt.show()

In [ ]:
preprocess_input = keras.applications.mobilenet_v2.preprocess_input
decode_predictions = keras.applications.mobilenet_v2.decode_predictions

last_conv_layer_name = 'conv2d_93'

# Remove-se o softmax da última camada
model.layers[-1].activation = None

In [ ]:
# Exibe a parte das imagens usadas pela rede neural para classificar as imagens
fig, axes = plt.subplots(nrows = 4, ncols = 4, figsize = (20, 20), subplot_kw = {'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    img_array = X_test_doente[i]
    heatmap = make_gradcam_heatmap(img_array.reshape(1, 256, 256, 3), model, last_conv_layer_name)
    cam_path = save_and_display_gradcam(img_array, heatmap, "cam.jpg", 0.8)
    class_test = classes[y_test_doente2[i]]
    class_prediction = classes[y_pred_doente[i]]
    ax.imshow(plt.imread(cam_path))
    ax.set_title(f"True: {class_test}\nPredicted: {class_prediction}", fontsize = 14)
plt.tight_layout()
plt.show()

## **Cross Validation (Validação Cruzada)**

O objetivo da validação cruzada é o particionamento do conjunto de dados em subconjuntos exclusivos, para posteriormente utilizar alguns dos subconjuntos, dados de treinamento, para a estimação dos parâmetros do modelo, sendo os subconjuntos restantes, dados de validação e/ou de teste, empregados na validação do modelo.

In [ ]:
def cross_validation(model, x_res, epochs, batch_size, cv, mask):
  model = model_classifier(model)

  early_stopping = keras.callbacks.EarlyStopping(monitor='loss', patience = 3)
  reduce_lr =  keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001)
  learning_rate_reduction = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', patience=2, verbose=2, factor=0.5, min_lr=0.00001)
  checkpoint_path = 'training_1/model.ckpt'
  checkpoint_dir = os.path.dirname(checkpoint_path)
  model_check = keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True, monitor = 'val_accuracy', mode = 'max', save_best_only=True, verbose=1)
  cp_callback = [early_stopping, reduce_lr , learning_rate_reduction, model_check]

  accuracy = []
  precision = []
  recall = []
  f1 = []
  for i in range(cv):
    X_train, X_val, X_test, X_train_mask, X_val_mask, X_test_mask, y_train, y_val, y_test = divide_base_date(x_res,0)

    # Etiquetas de codificação
    encoder = LabelEncoder()
    y_train = encoder.fit_transform(y_train)
    y_val = encoder.fit_transform(y_val)
    y_test = encoder.fit_transform(y_test)

    if mask:
      X_train = np.array(X_train_mask)
      X_val = np.array(X_val_mask)
      X_test = np.array(X_test_mask)
    else:
      X_train = np.array(X_train)
      X_val = np.array(X_val)
      X_test = np.array(X_test)

    classes = list(encoder.classes_)

    print(f"Iteração {i+1}/{cv}")
    model_history = model.fit(X_train, y_train, epochs = epochs, batch_size = batch_size, validation_data = (X_val, y_val), callbacks = cp_callback)

    prob_pred = model.predict(X_test)
    y_pred = np.argmax(prob_pred, axis = 1)

    accuracy.append(accuracy_score(y_test, y_pred))
    precision.append(precision_score(y_test, y_pred, average = 'weighted', zero_division = 1))
    recall.append(recall_score(y_test, y_pred, average = 'weighted', zero_division = 1))
    f1.append(f1_score(y_test, y_pred, average = 'weighted', zero_division = 1))

  # Métricas para avaliação do modelo
  test_accuracy = accuracy
  test_accuracy_mean = np.average(accuracy)*100
  test_precision = precision
  test_precision_mean = np.average(precision)
  test_recall = recall
  test_recall_mean = np.average(recall)
  test_f1 = f1
  test_f1_mean = np.average(f1)

  metrics = {"Validation Accuracy scores": test_accuracy,"Mean Validation Accuracy": test_accuracy_mean, "Validation Precision scores": test_precision,
          "Mean Validation Precision": test_precision_mean, "Validation Recall scores": test_recall, "Mean Validation Recall": test_recall_mean,
          "Validation F1 scores": test_f1, "Mean Validation F1 Score": test_f1_mean}

  return metrics

In [ ]:
cross_validation("vgg16", images, 30, 25, 5, True)